In [1]:
import { ArrayHelpers, DateTimeFormatWrapper } from "../utils/utils.ts";
import { getRecords } from "./transmission.ts";
import type { DateUploadSpeed, StatsRecord } from "./transmission.ts"

import { document } from "jsr:@ry/jupyter-helper";

import * as Plot from "npm:@observablehq/plot";

const RAW_RECORDS = await getRecords("transmission.csv");

const LOCALE = new Intl.Locale("fr-FR");

const uploadSpeedFormat = new Intl.NumberFormat(LOCALE, {
  notation: "compact",
  style: "unit",
  unit: "byte-per-second",
});

console.table(RAW_RECORDS);


┌───────┬──────────────────────────┬─────────────────┬────────────┬───────────────┬──────────────┬─────────────────┬────────────────────┬───────────────┬────────────────────┬──────────────┬─────────────┐
│ (idx) │ date                     │ downloadedBytes │ filesAdded │ secondsActive │ sessionCount │ uploadedBytes   │ activeTorrentCount │ downloadSpeed │ pausedTorrentCount │ torrentCount │ uploadSpeed │
├───────┼──────────────────────────┼─────────────────┼────────────┼───────────────┼──────────────┼─────────────────┼────────────────────┼───────────────┼────────────────────┼──────────────┼─────────────┤
│     0 │ 2025-07-16T21:11:27.759Z │   3925170815043 │     237665 │      57124256 │          708 │ 171586887710452 │                183 │             0 │                255 │          438 │     1000000 │
│     1 │ 2025-07-16T21:11:40.640Z │   3925170815043 │     237665 │      57124269 │          708 │ 171586900710452 │                183 │             0 │                255 │          

In [2]:
const byHours = new ArrayHelpers(RAW_RECORDS)
  .map((v) => {
    const date = new Date(v.date);
    date.setMinutes(0, 0, 0);
    return {
      ...v,
      date,
    };
  })
  .groupBy((v) => v.date.getTime())
  .values()
  .map<DateUploadSpeed>((values) => {
    const dest: DateUploadSpeed = Object.create(null);
    dest.date = values.getFirst().date;
    dest.uploadSpeed = Math.round(values.average(e => e.uploadSpeed));
    return dest;
  })
  .toArray();
const byHoursFormatter = new DateTimeFormatWrapper(LOCALE, {
  weekday: "short",
  day: "numeric",
  hour: "numeric",
  hour12: false,
});

Plot.plot({
  document,
  title: "Vitesse d’envoi par heure",
  x: {
    tickFormat: byHoursFormatter.tickFormat((m) =>
      `${m.get("weekday")} ${m.get("day")}\n${m.get("hour")}h`
    ),
  },
  marks: [
    Plot.barY(byHours, {
      x: "date",
      y: "uploadSpeed",
      margin: 50,
    }),
    Plot.line(byHours, {
      x: "date",
      y: "uploadSpeed",
      stroke: "red",
      curve: "natural",
      margin: 50,
      marker: "dot",
    }),
    Plot.text(byHours, {
      x: "date",
      y: "uploadSpeed",
      text: (d) => uploadSpeedFormat.format(d.uploadSpeed),
      dy: -10,
    }),
  ],
  // margin: moy.length * 3,
  width: byHours.length * 50,
  figure: true,
});


Vitesse d’envoi par heure 0 2,000,000 4,000,000 6,000,000 8,000,000 10,000,000 12,000,000 14,000,000 ↑ uploadSpeed mer. 16 23h jeu. 17 00h jeu. 17 01h jeu. 17 02h jeu. 17 03h jeu. 17 04h jeu. 17 05h jeu. 17 06h jeu. 17 07h jeu. 17 08h jeu. 17 09h jeu. 17 10h jeu. 17 11h jeu. 17 12h jeu. 17 13h jeu. 17 14h jeu. 17 15h jeu. 17 16h jeu. 17 17h jeu. 17 18h jeu. 17 19h jeu. 17 20h jeu. 17 21h jeu. 17 22h jeu. 17 23h ven. 18 00h ven. 18 01h ven. 18 02h ven. 18 03h ven. 18 04h ven. 18 05h ven. 18 06h ven. 18 07h ven. 18 08h ven. 18 09h ven. 18 10h ven. 18 11h ven. 18 12h ven. 18 13h ven. 18 14h ven. 18 15h ven. 18 16h ven. 18 17h ven. 18 18h ven. 18 19h ven. 18 20h ven. 18 21h ven. 18 22h ven. 18 23h sam. 19 00h sam. 19 01h sam. 19 02h sam. 19 03h sam. 19 04h sam. 19 05h sam. 19 06h sam. 19 07h sam. 19 08h sam. 19 09h sam. 19 10h sam. 19 11h sam. 19 12h sam. 19 13h sam. 19 14h sam. 19 15h sam. 19 16h sam. 19 17h sam. 19 18h sam. 19 19h sam. 19 20h sam. 19 21h sam. 19 22h sam. 19 23h dim. 20 00h dim. 20 01h dim. 20 02h dim. 20 03h dim. 20 04h dim. 20 05h dim. 20 06h dim. 20 07h dim. 20 08h dim. 20 09h dim. 20 10h dim. 20 11h dim. 20 12h dim. 20 13h dim. 20 14h dim. 20 15h dim. 20 16h dim. 20 17h dim. 20 18h dim. 20 19h dim. 20 20h dim. 20 21h dim. 20 22h dim. 20 23h lun. 21 00h lun. 21 01h lun. 21 02h lun. 21 03h lun. 21 04h lun. 21 05h lun. 21 06h lun. 21 07h lun. 21 08h lun. 21 09h lun. 21 10h lun. 21 11h lun. 21 12h lun. 21 13h lun. 21 14h lun. 21 15h lun. 21 16h lun. 21 17h lun. 21 18h lun. 21 19h lun. 21 20h lun. 21 21h lun. 21 22h lun. 21 23h mar. 22 00h mar. 22 01h mar. 22 02h mar. 22 03h mar. 22 04h mar. 22 05h mar. 22 06h mar. 22 07h mar. 22 08h mar. 22 09h mar. 22 10h mar. 22 11h mar. 22 12h mar. 22 13h mar. 22 14h mar. 22 15h mar. 22 16h mar. 22 17h mar. 22 18h mar. 22 19h mar. 22 20h mar. 22 21h mar. 22 22h mar. 22 23h mer. 23 00h mer. 23 01h mer. 23 02h mer. 23 03h mer. 23 04h mer. 23 05h mer. 23 06h mer. 23 07h mer. 23 08h mer. 23 09h mer. 23 10h mer. 23 11h mer. 23 12h mer. 23 13h mer. 23 14h mer. 23 15h mer. 23 16h mer. 23 17h mer. 23 18h mer. 23 19h mer. 23 20h mer. 23 21h mer. 23 22h mer. 23 23h jeu. 24 00h jeu. 24 01h jeu. 24 02h jeu. 24 03h jeu. 24 04h jeu. 24 05h jeu. 24 06h jeu. 24 07h jeu. 24 08h jeu. 24 09h jeu. 24 10h jeu. 24 11h jeu. 24 12h jeu. 24 13h jeu. 24 14h jeu. 24 15h jeu. 24 16h jeu. 24 17h jeu. 24 18h jeu. 24 19h jeu. 24 20h jeu. 24 21h jeu. 24 22h jeu. 24 23h ven. 25 00h ven. 25 01h ven. 25 02h ven. 25 03h ven. 25 04h ven. 25 05h ven. 25 06h ven. 25 07h ven. 25 08h ven. 25 09h ven. 25 10h ven. 25 11h ven. 25 12h ven. 25 13h ven. 25 14h ven. 25 15h ven. 25 16h ven. 25 17h ven. 25 18h ven. 25 20h ven. 25 21h ven. 25 22h ven. 25 23h sam. 26 00h sam. 26 01h sam. 26 02h sam. 26 03h sam. 26 04h sam. 26 05h sam. 26 06h sam. 26 07h sam. 26 08h sam. 26 09h sam. 26 10h sam. 26 11h sam. 26 12h sam. 26 13h sam. 26 14h sam. 26 15h sam. 26 16h sam. 26 17h sam. 26 18h sam. 26 19h sam. 26 20h sam. 26 21h sam. 26 22h sam. 26 23h dim. 27 00h dim. 27 01h dim. 27 02h dim. 27 03h dim. 27 04h dim. 27 05h dim. 27 06h dim. 27 07h dim. 27 08h dim. 27 09h dim. 27 10h dim. 27 11h dim. 27 12h dim. 27 13h dim. 27 14h dim. 27 15h dim. 27 16h dim. 27 17h dim. 27 18h dim. 27 19h dim. 27 20h dim. 27 21h dim. 27 22h dim. 27 23h lun. 28 00h <path d="M54.96,329.705C71.495,326.613,88.03,323.52,104.564,325.254C121.099,326.988,137.634,333.548,154.168,331.912C170.703,330.276,187.238,320.444,203.772,314.131C220.307,307.817,236.841,305.021,253.376,304.943C269.911,304.865,286.445,307.505,302.98,314.505C319.515,321.506,336.049,332.868,352.584,334.466C369.119,336.064,385.653,327.899,402.188,323.249C418.722,318.6,435.257,317.467,451.792,324.017C468.326,330.568,484.861,344.801,501.396,350C517.93,355.199,534.465,351.363,551,350C567.534,348.637,584.069,349.748,600.604,346.88C617.138,344.012,633.673,337.165,650.207,336.514C666.742,335.862,683.277,341.405,699.811,342.79C716.346,344.175,732.881,341.402,749.415,326.792C765.95,312.182,782.485,285.735,

In [3]:
const moyByHours = new ArrayHelpers(RAW_RECORDS)
  .map((v) => {
    const date = new Date(0);
    date.setHours(v.date.getHours());
    return {
      ...v,
      date,
    };
  })
  .groupBy(({ date }) => date.getTime())
  .values()
  .map<DateUploadSpeed>((values) => {
    const dest: DateUploadSpeed = Object.create(null);
    dest.date = values.getFirst().date;
    dest.uploadSpeed = Math.round(values.average(v => v.uploadSpeed));
    return dest;
  })
  .toArray()
  .toSorted((a, b) => a.date.getTime() - b.date.getTime());

const moyByHoursFormatter = new DateTimeFormatWrapper(LOCALE, {
  hour: "2-digit",
  hour12: false,
});

Plot.plot({
  title: "Moyenne de vitesse d’envoi par heure",
  document,
  x: {
    tickFormat: moyByHoursFormatter.tickFormat((m) => `${m.get("hour")}h`),
  },
  marks: [
    Plot.barY(moyByHours, {
      x: "date",
      y: "uploadSpeed",
    }),
    Plot.line(moyByHours, {
      x: "date",
      y: "uploadSpeed",
      stroke: "red",
      curve: "natural",
      marker: "dot",
    }),
    Plot.text(moyByHours, {
      x: "date",
      y: "uploadSpeed",
      text: (d) => uploadSpeedFormat.format(d.uploadSpeed),
      dy: -10,
    }),
  ],
  margin: 50,
  // margin: moy.length * 3,
  width: moyByHours.length * 50,
  figure: true,
});


Moyenne de vitesse d’envoi par heure 0 200,000 400,000 600,000 800,000 1,000,000 1,200,000 1,400,000 1,600,000 1,800,000 2,000,000 2,200,000 2,400,000 2,600,000 ↑ uploadSpeed 00h 01h 02h 03h 04h 05h 06h 07h 08h 09h 10h 11h 12h 13h 14h 15h 16h 17h 18h 19h 20h 21h 22h 23h <path d="M62,187.471C77,197.071,92,206.672,107,177.395C122,148.119,137,79.965,152,84.722C167,89.479,182,167.147,197,173.261C212,179.376,227,113.938,242,133.44C257,152.943,272,257.387,287,275.788C302,294.189,317,226.547,332,188.193C347,149.839,362,140.773,377,188.237C392,235.701,407,339.694,422,383.138C437,426.582,452,409.476,467,395.374C482,381.271,497,370.171,512,346.535C527,322.9,542,286.729,557,264.72C572,242.711,587,234.864,602,233.86C617,232.857,632,238.697,647,234.901C662,231.106,677,217.675,692,217.668C707,217.662,722,231.081,737,227.688C752,224.296,767,204.093,782,207.527C797,210.962,812,238.034,827,207.09C842,176.146,857,87.187,872,50C887,12.813,902,27.398,917,50.404C932,73.41,947,104.837,962,107.484C977,110.132,992,83.999,1007,99.015C1022,114.032,1037,170.196,1052,174.154C1067,178.113,1082,129.865,1097,81.618" marker-end="url(#plot-marker-2)" marker-mid="url(#plot-marker-2)" marker-start="url(#plot-marker-2)" /> 1,7 M o/s 1,8 M o/s 2,5 M o/s 1,8 M o/s 2,1 M o/s 977 k o/s 1,7 M o/s 1,7 M o/s 133 k o/s 36 k o/s 421 k o/s 1,1 M o/s 1,3 M o/s 1,3 M o/s 1,4 M o/s 1,4 M o/s 1,5 M o/s 1,5 M o/s 2,8 M o/s 2,8 M o/s 2,3 M o/s 2,4 M o/s 1,8 M o/s 2,5 M o/s ⚠️ 1 warning. Please check the console.